In [1]:
import numpy as np
import cv2
import scipy
import scipy.io
import sklearn
from tqdm import tqdm
from numpy import random

ModuleNotFoundError: No module named 'cv2'

In [6]:
salinas = scipy.io.loadmat('datasets/training/img')
X = salinas['img/img']
Y = salinas['mask/img']

AttributeError: module 'scipy.io' has no attribute 'load'

In [7]:
ica = FastICA(n_components=3, random_state=0)
X_ica = ica.fit_transform(X)
X_ica.shape

NameError: name 'FastICA' is not defined

In [4]:
X_ica = np.reshape(X_ica, [217,512,3])
X = np.zeros([512,217,3])
X[:,:,0] = np.transpose(X_ica[:,:,0])
X[:,:,1] = np.transpose(X_ica[:,:,1])
X[:,:,2] = np.transpose(X_ica[:,:,2])
Y = np.transpose(np.reshape(Y,[217,512]))

In [5]:
X.shapeJe 

(512, 217, 3)

In [6]:
X[:,:,0] = X[:,:,0]/np.max(X[:,:,0])
X[:,:,1] = X[:,:,1]/np.max(X[:,:,1])
X[:,:,2] = X[:,:,2]/np.max(X[:,:,2])
np.max(X[:,:,2])

1.0

In [7]:
cv2.imwrite(('../Desktop/%s.png' % 'Y'), Y*15)

True

In [8]:
img_w = 64 
img_h = 64  
 
image_sets = X_ica
    
def rotate(xb,yb,angle):
    M_rotate = cv2.getRotationMatrix2D((img_w/2, img_h/2), angle, 1)
    xb = cv2.warpAffine(xb, M_rotate, (img_w, img_h))
    yb = cv2.warpAffine(yb, M_rotate, (img_w, img_h))
    return xb,yb
    
def blur(img):
    img = cv2.blur(img, (3, 3));
    return img
 
def add_noise(img):
    for i in range(25):
        temp_x = np.random.randint(0,img.shape[0])
        temp_y = np.random.randint(0,img.shape[1])
        img[temp_x][temp_y] = 1
    return img

In [9]:
def data_augment(xb,yb):
    if np.random.random() < 0.25:
        xb,yb = rotate(xb,yb,90)
    if np.random.random() < 0.25:
        xb,yb = rotate(xb,yb,180)
    if np.random.random() < 0.25:
        xb,yb = rotate(xb,yb,270)
    if np.random.random() < 0.25:
        xb = cv2.flip(xb, 1)  # flipcode > 0：沿y轴翻转
        yb = cv2.flip(yb, 1)
        
    if np.random.random() < 0.25:
        xb = blur(xb)
    
    if np.random.random() < 0.2:
        xb = add_noise(xb)
        
    return xb,yb

In [10]:
def creat_dataset(image_num, mode):
    print('creating dataset...')
    g_count = 0
#    for i in tqdm(range(1)):
    count = 0
        #src_img = cv2.imread('./data/src/' + image_sets[i])  # 3 channels
        #label_img = cv2.imread('./data/label/' + image_sets[i],cv2.IMREAD_GRAYSCALE)  # single channel
    src_img = X*255
    label_img = Y
    X_height,X_width,_ = src_img.shape
    while count < image_num:
        random_width = random.randint(0, X_width - img_w - 1)
        random_height = random.randint(0, X_height - img_h - 1)
        src_roi = src_img[random_height: random_height + img_h, random_width: random_width + img_w,:]
        label_roi = label_img[random_height: random_height + img_h, random_width: random_width + img_w]
        if mode == 'augment':
            src_roi,label_roi = data_augment(src_roi,label_roi)
            
        visualize = np.zeros((32,32)).astype(np.uint8)
        visualize = label_roi *16-1
            
        cv2.imwrite(('../Desktop/Train/visualize/%d.png' % g_count),visualize)
        cv2.imwrite(('../Desktop/Train/src/%d.png' % g_count),src_roi)
        cv2.imwrite(('../Desktop/Train/label/%d.png' % g_count),label_roi)
        count += 1 
        g_count += 1

In [11]:
creat_dataset(2000, 'augment')

creating dataset...


In [53]:
from keras.datasets import mnist
from keras.preprocessing.image import load_img, array_to_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten,Dense, BatchNormalization, UpSampling2D, Reshape, Permute, Activation
from keras.callbacks import ModelCheckpoint

import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [63]:
def SegNet():  
    model = Sequential()  
    #encoder  
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(img_w,img_h,3),padding='same',activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu'))  
    model.add(BatchNormalization())  
    model.add(MaxPooling2D(pool_size=(2,2)))  
    #(128,128)  
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    #(64,64)  
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    #(32,32)  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    #(16,16)  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    #(8,8)  
    #decoder  
    model.add(UpSampling2D(size=(2,2)))  
    #(16,16)  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(UpSampling2D(size=(2, 2)))  
    #(32,32)  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(UpSampling2D(size=(2, 2)))  
    #(64,64)  
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(UpSampling2D(size=(2, 2)))  
    #(128,128)  
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(UpSampling2D(size=(2, 2)))  
    #(256,256)  
    model.add(Conv2D(64, (3, 3), strides=(1, 1), input_shape=(3,img_w, img_h), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))  
    model.add(BatchNormalization())  
    model.add(Conv2D(n_label, (1, 1), strides=(1, 1), padding='same'))  
    model.add(Reshape((n_label,img_w*img_h)))  
    #axis=1和axis=2互换位置，等同于np.swapaxes(layer,1,2)  
    model.add(Permute((2,1)))  
    model.add(Activation('softmax'))  
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
    model.summary()  
    return model 

In [59]:
def get_train_val(val_rate = 0.25):
    train_url = []    
    train_set = []
    val_set  = []
    for pic in os.listdir(filepath + 'src'):
        train_url.append(pic)
    random.shuffle(train_url)
    total_num = len(train_url)
    val_num = int(val_rate * total_num)
    for i in range(len(train_url)):
        if i < val_num:
            val_set.append(train_url[i]) 
        else:
            train_set.append(train_url[i])
    return train_set,val_set
    
# data for training  
def generateData(batch_size,data=[]):  
    #print 'generateData...'
    while True:  
        train_data = []  
        train_label = []  
        batch = 0  
        for i in (range(len(data))): 
            url = data[i]
            batch += 1 
            #print (filepath + 'src/' + url)
            #img = load_img(filepath + 'src/' + url, target_size=(img_w, img_h))  
            img = load_img(filepath + 'src/' + url)
            img = img_to_array(img) 
            # print img
            # print img.shape  
            train_data.append(img)  
            #label = load_img(filepath + 'label/' + url, target_size=(img_w, img_h),grayscale=True)
            label = load_img(filepath + 'label/' + url, grayscale=True)
            label = img_to_array(label).reshape((img_w * img_h,))  
            # print label.shape  
            train_label.append(label)  
            if batch % batch_size==0: 
                #print 'get enough bacth!\n'
                train_data = np.array(train_data)  
                train_label = np.array(train_label).flatten()  
                #train_label = LabelEncoder.fit_transform(train_label)  
                train_label = to_categorical(train_label, num_classes=n_label)  
                train_label = train_label.reshape((batch_size,img_w * img_h,n_label))  
                yield (train_data,train_label)  
                train_data = []  
                train_label = []  
                batch = 0  

# data for validation 
def generateValidData(batch_size,data=[]):  
    #print 'generateValidData...'
    while True:  
        valid_data = []  
        valid_label = []  
        batch = 0  
        for i in (range(len(data))):  
            url = data[i]
            batch += 1  
            #img = load_img(filepath + 'src/' + url, target_size=(img_w, img_h))
            img = load_img(filepath + 'src/' + url)
            #print img
            #print (filepath + 'src/' + url)
            img = img_to_array(img)  
            # print img.shape  
            valid_data.append(img)  
            #label = load_img(filepath + 'label/' + url, target_size=(img_w, img_h),grayscale=True)
            label = load_img(filepath + 'label/' + url, grayscale=True)
            label = img_to_array(label).reshape((img_w * img_h,))  
            # print label.shape  
            valid_label.append(label)  
            if batch % batch_size==0:  
                valid_data = np.array(valid_data)  
                valid_label = np.array(valid_label).flatten()  
                #valid_label = LabelEncoder.fit_transform(valid_label)  
                valid_label = to_categorical(valid_label, num_classes=n_label)  
                valid_label = valid_label.reshape((batch_size,img_w * img_h,n_label))  
                yield (valid_data,valid_label)  
                valid_data = []  
                valid_label = []  
                batch = 0

In [69]:
EPOCHS = 30
BS = 15
n_label = 17
args = {'model':[]}
filepath = '../Desktop/Train/'
model = SegNet()  
modelcheck = ModelCheckpoint(args['model'],monitor='val_acc',save_best_only=True,mode='max')  
callable = [modelcheck]  
train_set,val_set = get_train_val()
train_numb = len(train_set)  
valid_numb = len(val_set)  
print ("the number of train data is",train_numb)  
print ("the number of val data is",valid_numb)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_601 (Conv2D)          (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization_581 (Bat (None, 64, 64, 64)        256       
_________________________________________________________________
conv2d_602 (Conv2D)          (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_582 (Bat (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_118 (MaxPoolin (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_603 (Conv2D)          (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_583 (Bat (None, 32, 32, 128)     

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=20, 
                    validation_data=(X_test, y_test))

In [73]:
H = model.fit(generateData(BS,train_set),steps_per_epoch=train_numb//BS,epochs=EPOCHS,
              validation_data=generateValidData(BS,val_set),validation_steps=valid_numb//BS)  

Epoch 1/30
100/100 [==============================] - 152s 2s/step - loss: 1.1219 - accuracy: 0.5963 - val_loss: 1.0643 - val_accuracy: 0.6253
Epoch 2/30
100/100 [==============================] - 153s 2s/step - loss: 0.9920 - accuracy: 0.6311 - val_loss: 1.0047 - val_accuracy: 0.6387
Epoch 3/30
100/100 [==============================] - 153s 2s/step - loss: 0.9069 - accuracy: 0.6548 - val_loss: 0.9555 - val_accuracy: 0.6475
Epoch 4/30
100/100 [==============================] - 153s 2s/step - loss: 0.8590 - accuracy: 0.6646 - val_loss: 0.9561 - val_accuracy: 0.6498
Epoch 5/30
100/100 [==============================] - 155s 2s/step - loss: 0.8147 - accuracy: 0.6756 - val_loss: 0.9063 - val_accuracy: 0.6602
Epoch 6/30
100/100 [==============================] - 155s 2s/step - loss: 0.7825 - accuracy: 0.6829 - val_loss: 0.8699 - val_accuracy: 0.6641
Epoch 7/30
100/100 [==============================] - 154s 2s/step - loss: 0.7545 - accuracy: 0.6884 - val_loss: 0.8574 - val_accuracy: 0.6714

In [67]:

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on SegNet Satellite Seg")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

SyntaxError: invalid syntax (<ipython-input-67-40005d6026a4>, line 2)